In [1]:
import tensorflow as tf

In [2]:
## changing the width of cells
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

## Create a graph

In [3]:
a = tf.constant(3, name = "a")
b = tf.constant(-1, name = "b")

In [4]:
x = tf.Variable(1, name = "x")

We use `tf.identity` to be able to assign a name to the output of our computation, so that we can later "fetch" it in java:

In [5]:
f = tf.identity(a * x + b, name = "f")

## Run it in a session

In [6]:
sess = tf.InteractiveSession()

In [7]:
sess.run(tf.global_variables_initializer())

In [8]:
print(sess.run(f, feed_dict={x: 2}))

5


## Saving

In [11]:
builder = tf.saved_model.builder.SavedModelBuilder('d:/tmp/model03')

In [12]:
builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING])

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.


In [13]:
builder.save() 

INFO:tensorflow:SavedModel written to: b'd:/tmp/model03\\saved_model.pb'


b'd:/tmp/model03\\saved_model.pb'

## Calling it from Java
```java
import org.tensorflow.Graph;
import org.tensorflow.SavedModelBundle;
import org.tensorflow.Session;
import org.tensorflow.Session.Runner;
import org.tensorflow.Tensor;
import org.tensorflow.TensorFlow;

public class ImportModel_01 {
	
	/*
	 * This block is here to import the tensorflow dll.
	 * There are probably better ways to do it : setting a Path variable 
	 * so that java can find it automaticaly. 
	 */
	static {
	    try {
	    	System.load("C:/Users/sakalos/moje/java_libs/tensorflow/libtensorflow_jni-cpu-windows-x86_64-1.1.0/tensorflow_jni.dll");
	    } catch (UnsatisfiedLinkError e) {
	      System.err.println("TensorFlow dll file failed to load.\n" + e);
	      System.exit(1);
	    }
	  }
	
  public static void main(String[] args) throws Exception {
	  try (SavedModelBundle bundle = SavedModelBundle.load("d:/tmp/model01", "serve")) {
		  Session sess = bundle.session();
		  Tensor a = sess.runner().fetch("a").run().get(0);
		  Tensor b = sess.runner().fetch("b").run().get(0);
		  System.out.println(
				  "a = " + a.intValue());
		  System.out.println(
				  "b = " + b.intValue());
		  
		  Runner f = sess.runner().fetch("f");
		  try(Tensor x_val = Tensor.create( 4 )) {
			  System.out.println(
					  "x_val = " + x_val.intValue());
			  System.out.println(
					  "f(x) = " + f.feed("x", x_val).run().get(0).intValue()
					);
		  }
		  
	}
  }
}
```